![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/MEDICAL_LLM.ipynb)

# **MEDICAL_LLM**

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [ ]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 4.4.4
Spark NLP_JSL Version : 4.4.4


# **Medical Question Answering**

**Welcoming BioGPT (Generative Pre-Trained Transformer For Biomedical Text Generation and Mining) to Spark NLP**

BioGPT is a domain-specific generative pre-trained Transformer language model for biomedical text generation and mining. BioGPT follows the Transformer language model backbone, and is pre-trained on 15M PubMed abstracts from scratch. Experiments demonstrate that BioGPT achieves better performance compared with baseline methods and other well-performing methods across all the tasks. Read more at [the official paper.](https://arxiv.org/abs/2210.10341)

We ported BioGPT (`BioGPT-QA-PubMedQA-BioGPT`) into Spark NLP for Healthcare with better inference speed and memory optimization, that enable us to get 20x faster results with less memory compared to the HuggingFace version.


# 🔎 MODELS

<div align="center">

| **Index** | **Question Answer Models**        |
|---------------|----------------------|
| 1        | [medical_qa_biogpt](https://nlp.johnsnowlabs.com/2023/05/17/medical_qa_biogpt_en.html)     |
| 2          | [flan_t5_base_jsl_qa](https://nlp.johnsnowlabs.com/2023/05/15/flan_t5_base_jsl_qa_en.html)       |
| 3          | [clinical_notes_qa_large](https://nlp.johnsnowlabs.com/2023/07/06/clinical_notes_qa_base.html)       |
| 4          | [clinical_notes_qa_base](https://nlp.johnsnowlabs.com/2023/07/06/clinical_notes_qa_large.html)       |


</div>

In [ ]:
paper_abstract = [
    "We have previously reported the feasibility of diagnostic and therapeutic peritoneoscopy including liver biopsy, gastrojejunostomy, and tubal ligation by an oral transgastric approach. We present results of per-oral transgastric splenectomy in a porcine model. The goal of this study was to determine the technical feasibility of per-oral transgastric splenectomy using a flexible endoscope. We performed acute experiments on 50-kg pigs. All animals were fed liquids for 3 days prior to procedure. The procedures were performed under general anesthesia with endotracheal intubation. The flexible endoscope was passed per orally into the stomach and puncture of the gastric wall was performed with a needle knife. The puncture was extended to create a 1.5-cm incision using a pull-type sphincterotome, and a double-channel endoscope was advanced into the peritoneal cavity. The peritoneal cavity was insufflated with air through the endoscope. The spleen was visualized. The splenic vessels were ligated with endoscopic loops and clips, and then mesentery was dissected using electrocautery. Endoscopic splenectomy was performed on six pigs. There were no complications during gastric incision and entrance into the peritoneal cavity. Visualization of the spleen and other intraperitoneal organs was very good. Ligation of the splenic vessels and mobilization of the spleen were achieved using commercially available devices and endoscopic accessories.",
    "To evaluate the degree to which histologic chorioamnionitis, a frequent finding in placentas submitted for histopathologic evaluation, correlates with clinical indicators of infection in the mother. A retrospective review was performed on 52 cases with a histologic diagnosis of acute chorioamnionitis from 2,051 deliveries at University Hospital, Newark, from January 2003 to July 2003. Third-trimester placentas without histologic chorioamnionitis (n = 52) served as controls. Cases and controls were selected sequentially. Maternal medical records were reviewed for indicators of maternal infection. Histologic chorioamnionitis was significantly associated with the usage of antibiotics (p = 0.0095) and a higher mean white blood cell count (p = 0.018). The presence of 1 or more clinical indicators was significantly associated with the presence of histologic chorioamnionitis (p = 0.019).",
    "In patients with Los Angeles (LA) grade C or D oesophagitis, a positive relationship has been established between the duration of intragastric acid suppression and healing.AIM: To determine whether there is an apparent optimal time of intragastric acid suppression for maximal healing of reflux oesophagitis. Post hoc analysis of data from a proof-of-concept, double-blind, randomized study of 134 adult patients treated with esomeprazole (10 or 40 mg od for 4 weeks) for LA grade C or D oesophagitis. A curve was fitted to pooled 24-h intragastric pH (day 5) and endoscopically assessed healing (4 weeks) data using piecewise quadratic logistic regression. Maximal reflux oesophagitis healing rates were achieved when intragastric pH>4 was achieved for approximately 50-70% (12-17 h) of the 24-h period. Acid suppression above this threshold did not yield further increases in healing rates."
                  ]

question = [
    "Transgastric endoscopic splenectomy: is it possible?",
    "Does histologic chorioamnionitis correspond to clinical chorioamnionitis?",
    "Is there an optimal time of acid suppression for maximal healing?"
          ]

data = spark.createDataFrame(
    [
        [paper_abstract[0],  question[0]],
        [paper_abstract[1],  question[1]],
        [paper_abstract[2],  question[2]],
    ]
).toDF("context","question")

data.show(truncate = 60)

+------------------------------------------------------------+------------------------------------------------------------+
|                                                     context|                                                    question|
+------------------------------------------------------------+------------------------------------------------------------+
|We have previously reported the feasibility of diagnostic...|        Transgastric endoscopic splenectomy: is it possible?|
|To evaluate the degree to which histologic chorioamnionit...|Does histologic chorioamnionitis correspond to clinical c...|
|In patients with Los Angeles (LA) grade C or D oesophagit...|Is there an optimal time of acid suppression for maximal ...|
+------------------------------------------------------------+------------------------------------------------------------+



## medical_qa_biogpt

### **Short answer**

 `Short` answers usually consist of brief and concise responses such as "yes" or "no", and they are typically given as yes/no responses to do/does, is/are, or other question words.

In [ ]:
document_assembler = MultiDocumentAssembler()\
    .setInputCols("question", "context")\
    .setOutputCols("document_question", "document_context")

med_qa = MedicalQuestionAnswering.pretrained("medical_qa_biogpt","en","clinical/models")\
    .setInputCols(["document_question", "document_context"])\
    .setOutputCol("answer")\
    .setMaxNewTokens(30)\
    .setTopK(1)\
    .setQuestionType("short") # "long"

pipeline = Pipeline(stages=[document_assembler, med_qa])

empty_data = spark.createDataFrame([["",""]]).toDF("question", "context")

model = pipeline.fit(empty_data)

medical_qa_biogpt download started this may take some time.
[OK!]


In [ ]:
model.transform(data)\
  .selectExpr("document_question.result as Question", "answer.result as Short_Answer")\
  .show(truncate=False)

+---------------------------------------------------------------------------+------------+
|Question                                                                   |Short_Answer|
+---------------------------------------------------------------------------+------------+
|[Transgastric endoscopic splenectomy: is it possible?]                     |[yes]       |
|[Does histologic chorioamnionitis correspond to clinical chorioamnionitis?]|[yes]       |
|[Is there an optimal time of acid suppression for maximal healing?]        |[no]        |
+---------------------------------------------------------------------------+------------+



**LightPipelines**

In [ ]:
context ="We have previously reported the feasibility of diagnostic and therapeutic peritoneoscopy including liver biopsy, gastrojejunostomy, and tubal ligation by an oral transgastric approach. We present results of per-oral transgastric splenectomy in a porcine model. The goal of this study was to determine the technical feasibility of per-oral transgastric splenectomy using a flexible endoscope. We performed acute experiments on 50-kg pigs. All animals were fed liquids for 3 days prior to procedure. The procedures were performed under general anesthesia with endotracheal intubation. The flexible endoscope was passed per orally into the stomach and puncture of the gastric wall was performed with a needle knife. The puncture was extended to create a 1.5-cm incision using a pull-type sphincterotome, and a double-channel endoscope was advanced into the peritoneal cavity. The peritoneal cavity was insufflated with air through the endoscope. The spleen was visualized. The splenic vessels were ligated with endoscopic loops and clips, and then mesentery was dissected using electrocautery. Endoscopic splenectomy was performed on six pigs. There were no complications during gastric incision and entrance into the peritoneal cavity. Visualization of the spleen and other intraperitoneal organs was very good. Ligation of the splenic vessels and mobilization of the spleen were achieved using commercially available devices and endoscopic accessories."
question = "Transgastric endoscopic splenectomy: is it possible?"

light_model = LightPipeline(model)

light_result = light_model.annotate([question],[context])

light_result

[{'document_question': ['Transgastric endoscopic splenectomy: is it possible?'],
  'document_context': ['We have previously reported the feasibility of diagnostic and therapeutic peritoneoscopy including liver biopsy, gastrojejunostomy, and tubal ligation by an oral transgastric approach. We present results of per-oral transgastric splenectomy in a porcine model. The goal of this study was to determine the technical feasibility of per-oral transgastric splenectomy using a flexible endoscope. We performed acute experiments on 50-kg pigs. All animals were fed liquids for 3 days prior to procedure. The procedures were performed under general anesthesia with endotracheal intubation. The flexible endoscope was passed per orally into the stomach and puncture of the gastric wall was performed with a needle knife. The puncture was extended to create a 1.5-cm incision using a pull-type sphincterotome, and a double-channel endoscope was advanced into the peritoneal cavity. The peritoneal cavity 

In [ ]:
light_result[0]["answer"]

['yes']

### **Long answer**
A `long` answer is a response that goes into more detail and provides a more comprehensive explanation than a `short` answer. `Long` answers typically involve more elaboration, examples, and supporting evidence to provide a thorough understanding of a particular topic or question.

In [ ]:
document_assembler = MultiDocumentAssembler()\
    .setInputCols("question", "context")\
    .setOutputCols("document_question", "document_context")

med_qa = MedicalQuestionAnswering.pretrained("medical_qa_biogpt","en","clinical/models")\
    .setInputCols(["document_question", "document_context"])\
    .setOutputCol("answer")\
    .setMaxNewTokens(30)\
    .setTopK(1)\
    .setQuestionType("long") # "short"

pipeline = Pipeline(stages=[document_assembler, med_qa])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

medical_qa_biogpt download started this may take some time.
[OK!]


In [ ]:
model.transform(data)\
  .selectExpr("document_question.result as Question", "answer.result as Long_Answer")\
  .show(truncate=False)

+---------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Question                                                                   |Long_Answer                                                                                                                                                                                             |
+---------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[Transgastric endoscopic splenectomy: is it possible?]                     |[per - oral transgastric splenectomy was technically feasible in a porcine model. furt

**LightPipelines**

In [ ]:
context ="We have previously reported the feasibility of diagnostic and therapeutic peritoneoscopy including liver biopsy, gastrojejunostomy, and tubal ligation by an oral transgastric approach. We present results of per-oral transgastric splenectomy in a porcine model. The goal of this study was to determine the technical feasibility of per-oral transgastric splenectomy using a flexible endoscope. We performed acute experiments on 50-kg pigs. All animals were fed liquids for 3 days prior to procedure. The procedures were performed under general anesthesia with endotracheal intubation. The flexible endoscope was passed per orally into the stomach and puncture of the gastric wall was performed with a needle knife. The puncture was extended to create a 1.5-cm incision using a pull-type sphincterotome, and a double-channel endoscope was advanced into the peritoneal cavity. The peritoneal cavity was insufflated with air through the endoscope. The spleen was visualized. The splenic vessels were ligated with endoscopic loops and clips, and then mesentery was dissected using electrocautery. Endoscopic splenectomy was performed on six pigs. There were no complications during gastric incision and entrance into the peritoneal cavity. Visualization of the spleen and other intraperitoneal organs was very good. Ligation of the splenic vessels and mobilization of the spleen were achieved using commercially available devices and endoscopic accessories."
question = "Transgastric endoscopic splenectomy: is it possible?"

light_model = LightPipeline(model)

light_result = light_model.annotate([question],[context])

light_result

[{'document_question': ['Transgastric endoscopic splenectomy: is it possible?'],
  'document_context': ['We have previously reported the feasibility of diagnostic and therapeutic peritoneoscopy including liver biopsy, gastrojejunostomy, and tubal ligation by an oral transgastric approach. We present results of per-oral transgastric splenectomy in a porcine model. The goal of this study was to determine the technical feasibility of per-oral transgastric splenectomy using a flexible endoscope. We performed acute experiments on 50-kg pigs. All animals were fed liquids for 3 days prior to procedure. The procedures were performed under general anesthesia with endotracheal intubation. The flexible endoscope was passed per orally into the stomach and puncture of the gastric wall was performed with a needle knife. The puncture was extended to create a 1.5-cm incision using a pull-type sphincterotome, and a double-channel endoscope was advanced into the peritoneal cavity. The peritoneal cavity 

In [ ]:
light_result[0]["answer"]

['per - oral transgastric splenectomy was technically feasible in a porcine model. further studies are necessary to determine the safety and efficacy of this procedure in']

## clinical_notes_qa_base

In [ ]:
document_assembler = MultiDocumentAssembler()\
    .setInputCols("question", "context")\
    .setOutputCols("document_question", "document_context")

med_qa  = sparknlp_jsl.annotators.MedicalQuestionAnswering()\
    .pretrained("clinical_notes_qa_base", "en", "clinical/models")\
    .setInputCols(["document_question", "document_context"])\
    .setCustomPrompt("Context: {context} \n Question: {question} \n Answer: ")\
    .setOutputCol("answer")\

pipeline = Pipeline(stages=[document_assembler, med_qa])

note_text = "I am asked to see the patient today with ongoing issues around her diabetic control. We have been fairly aggressively, downwardly adjusting her insulins, both the Lantus insulin, which we had been giving at night as well as her sliding scale Humalog insulin prior to meals. Despite frequent decreases in her insulin regimen, she continues to have somewhat low blood glucoses, most notably in the morning when the glucoses have been in the 70s despite decreasing her Lantus insulin from around 84 units down to 60 units, which is a considerable change. What I cannot explain is why her glucoses have not really climbed at all despite the decrease in insulin. The staff reports to me that her appetite is good and that she is eating as well as ever. I talked to Anna today. She feels a little fatigued. Otherwise, she is doing well."

question = "How much has the Lantus insulin dosage been reduced?"

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

data = spark.createDataFrame([[question, note_text]]).toDF("question", "context")

result = model.transform(data)

clinical_notes_qa_base download started this may take some time.
[OK!]


In [ ]:
result.selectExpr("document_question.result as Question", "answer.result as Answer")\
  .show(truncate=False)

+------------------------------------------------------+-----------------------------------------------------------------------------------+
|Question                                              |Answer                                                                             |
+------------------------------------------------------+-----------------------------------------------------------------------------------+
|[How much has the Lantus insulin dosage been reduced?]|[The Lantus insulin dosage has been reduced from around 84 units down to 60 units.]|
+------------------------------------------------------+-----------------------------------------------------------------------------------+



In [ ]:
context ="I am asked to see the patient today with ongoing issues around her diabetic control. We have been fairly aggressively, downwardly adjusting her insulins, both the Lantus insulin, which we had been giving at night as well as her sliding scale Humalog insulin prior to meals. Despite frequent decreases in her insulin regimen, she continues to have somewhat low blood glucoses, most notably in the morning when the glucoses have been in the 70s despite decreasing her Lantus insulin from around 84 units down to 60 units, which is a considerable change. What I cannot explain is why her glucoses have not really climbed at all despite the decrease in insulin. The staff reports to me that her appetite is good and that she is eating as well as ever. I talked to Anna today. She feels a little fatigued. Otherwise, she is doing well."
question = "How much has the Lantus insulin dosage been reduced?"

light_model = LightPipeline(model)

light_result = light_model.annotate([question],[context])

light_result

[{'document_question': ['How much has the Lantus insulin dosage been reduced?'],
  'document_context': ['I am asked to see the patient today with ongoing issues around her diabetic control. We have been fairly aggressively, downwardly adjusting her insulins, both the Lantus insulin, which we had been giving at night as well as her sliding scale Humalog insulin prior to meals. Despite frequent decreases in her insulin regimen, she continues to have somewhat low blood glucoses, most notably in the morning when the glucoses have been in the 70s despite decreasing her Lantus insulin from around 84 units down to 60 units, which is a considerable change. What I cannot explain is why her glucoses have not really climbed at all despite the decrease in insulin. The staff reports to me that her appetite is good and that she is eating as well as ever. I talked to Anna today. She feels a little fatigued. Otherwise, she is doing well.'],
  'answer': ['The Lantus insulin dosage has been reduced from

## clinical_notes_qa_large

In [ ]:
document_assembler = MultiDocumentAssembler()\
    .setInputCols("question", "context")\
    .setOutputCols("document_question", "document_context")

med_qa  = sparknlp_jsl.annotators.MedicalQuestionAnswering()\
    .pretrained("clinical_notes_qa_large", "en", "clinical/models")\
    .setInputCols(["document_question", "document_context"])\
    .setCustomPrompt("Context: {context} \n Question: {question} \n Answer: ")\
    .setOutputCol("answer")\

pipeline = Pipeline(stages=[document_assembler, med_qa])

note_text = "his is a 14-month-old with history of chronic recurrent episodes of otitis media, totalling 6 bouts, requiring antibiotics since birth. There is also associated chronic nasal congestion. There had been no bouts of spontaneous tympanic membrane perforation, but there had been elevations of temperature up to 102 during the acute infection. He is being admitted at this time for myringotomy and tube insertion under general facemask anesthesia."

question = "How many bouts of otitis media has the patient experienced?"

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

data = spark.createDataFrame([[question, note_text]]).toDF("question", "context")

result = model.transform(data)

clinical_notes_qa_large download started this may take some time.
[OK!]


In [ ]:
result.selectExpr("document_question.result as Question", "answer.result as Answer")\
  .show(truncate=False)

+-------------------------------------------------------------+------------------------------------------------------+
|Question                                                     |Answer                                                |
+-------------------------------------------------------------+------------------------------------------------------+
|[How many bouts of otitis media has the patient experienced?]|[The patient has experienced 6 bouts of otitis media.]|
+-------------------------------------------------------------+------------------------------------------------------+



In [ ]:
context ="his is a 14-month-old with history of chronic recurrent episodes of otitis media, totalling 6 bouts, requiring antibiotics since birth. There is also associated chronic nasal congestion. There had been no bouts of spontaneous tympanic membrane perforation, but there had been elevations of temperature up to 102 during the acute infection. He is being admitted at this time for myringotomy and tube insertion under general facemask anesthesia."
question = "How many bouts of otitis media has the patient experienced?"

light_model = LightPipeline(model)

light_result = light_model.annotate([question],[context])

light_result

[{'document_question': ['How many bouts of otitis media has the patient experienced?'],
  'document_context': ['his is a 14-month-old with history of chronic recurrent episodes of otitis media, totalling 6 bouts, requiring antibiotics since birth. There is also associated chronic nasal congestion. There had been no bouts of spontaneous tympanic membrane perforation, but there had been elevations of temperature up to 102 during the acute infection. He is being admitted at this time for myringotomy and tube insertion under general facemask anesthesia.'],
  'answer': ['The patient has experienced 6 bouts of otitis media.']}]

# **Medical Text Summarization**

![IMAGE](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/databricks/python/data/Automated_Summarization_Clinical_Notes.png?raw=true)

🔎 Text Summarization is a natural language processing (NLP) task that involves condensing a lengthy text document into a shorter, more compact version while still retaining the most important information and meaning. The goal is to produce a summary that accurately represents the content of the original text in a concise form.

🔎There are `different approaches` to text summarization, including `extractive methods that` identify and extract important sentences or phrases from the text, and `abstractive methods` that generate new text based on the content of the original text.

![IMAGE](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/databricks/python/data/Summarization_Methods_vs_Quality_Dimensions.png?raw=true)

🔎 MedicalSummarizer annotator that uses a type of transformative model, the T5 model, to create a concise summary of medical text given in a clinical context. This annotator helps to quickly summarize complex medical information.

### 🔎 Models

<div align="center">

| **Index** | **Summarizer Models**        |
|---------------|----------------------|
| 1        | [summarizer_clinical_jsl](https://nlp.johnsnowlabs.com/2023/03/25/summarizer_clinical_jsl.html)     |
| 2          | [summarizer_clinical_jsl_augmented](https://nlp.johnsnowlabs.com/2023/03/30/summarizer_clinical_jsl_augmented_en.html)       |
| 3      | [summarizer_biomedical_pubmed](https://nlp.johnsnowlabs.com/2023/04/03/summarizer_biomedical_pubmed_en.html)    |
| 4      | [summarizer_generic_jsl](https://nlp.johnsnowlabs.com/2023/03/30/summarizer_generic_jsl_en.html)    |
| 5    | [summarizer_clinical_questions](https://nlp.johnsnowlabs.com/2023/04/03/summarizer_clinical_questions_en.html) |
| 6    | [summarizer_radiology](https://nlp.johnsnowlabs.com/2023/04/23/summarizer_jsl_radiology_en.html) |
| 7    | [summarizer_clinical_guidelines_large](https://nlp.johnsnowlabs.com/2023/05/08/summarizer_clinical_guidelines_large_en.html) |
| 8    | [summarizer_clinical_laymen](https://nlp.johnsnowlabs.com/2023/05/31/summarizer_clinical_laymen_en.html) |
</div>

## 📍Benchmark Report

Our clinical summarizer models with only 250M parameters perform 30-35% better than non-clinical SOTA text summarizers with 500M parameters, in terms of Bleu and Rouge benchmarks. That is, we achieve 30% better with half of the parameters that other LLMs have. See the details below.


### 🔎Benchmark on Samsum Dataset

| model_name | model_size | rouge | bleu | bertscore_precision | bertscore_recall: | bertscore_f1 |
|--|--|--|--|--|--|--|
philschmid/flan-t5-base-samsum | 240M | 0.2734 | 0.1813 | 0.8938 | 0.9133 | 0.9034 |
linydub/bart-large-samsum | 500M | 0.3060 | 0.2168 | 0.8961 | 0.9065 | 0.9013 |
philschmid/bart-large-cnn-samsum | 500M | 0.3794 | 0.1262 | 0.8599 | 0.9153 | 0.8867 |
transformersbook/pegasus-samsum | 570M | 0.3049 | 0.1543 | 0.8942 | 0.9183 | 0.9061 |
summarizer_generic_jsl | 240M | 0.2703 | 0.1932 | 0.8944 | 0.9161 | 0.9051 |


### 🔎Benchmark on MtSamples Summarization Dataset

| model_name | model_size | rouge | bleu | bertscore_precision | bertscore_recall: | bertscore_f1 |
|--|--|--|--|--|--|--|
philschmid/flan-t5-base-samsum | 250M | 0.1919 | 0.1124 | 0.8409 | 0.8964 | 0.8678 |
linydub/bart-large-samsum | 500M | 0.1586 | 0.0732 | 0.8747 | 0.8184 | 0.8456 |
philschmid/bart-large-cnn-samsum |  500M | 0.2170 | 0.1299 | 0.8846 | 0.8436 | 0.8636 |
transformersbook/pegasus-samsum | 500M | 0.1924 | 0.0965 | 0.8920 | 0.8149 | 0.8517 |
summarizer_clinical_jsl | 250M | 0.4836 | 0.4188 | 0.9041 | 0.9374 | 0.9204 |
summarizer_clinical_jsl_augmented | 250M | 0.5119 | 0.4545 | 0.9282 | 0.9526 | 0.9402 |


### 🔎Benchmark on MIMIC Summarization Dataset

| model_name | model_size | rouge | bleu | bertscore_precision | bertscore_recall: | bertscore_f1 |
|--|--|--|--|--|--|--|
philschmid/flan-t5-base-samsum | 250M | 0.1910 | 0.1037 | 0.8708 | 0.9056 | 0.8879 |
linydub/bart-large-samsum | 500M | 0.1252 | 0.0382 | 0.8933 | 0.8440 | 0.8679 |
philschmid/bart-large-cnn-samsum | 500M | 0.1795 | 0.0889 | 0.9172 | 0.8978 | 0.9074 |
transformersbook/pegasus-samsum | 570M | 0.1425 | 0.0582 | 0.9171 | 0.8682 | 0.8920 |
summarizer_clinical_jsl | 250M | 0.395 | 0.2962 | 0.895 | 0.9316 | 0.913 |
summarizer_clinical_jsl_augmented | 250M | 0.3964 | 0.307 | 0.9109 | 0.9452 | 0.9227 |

## 📃 summarizer_clinical_jsl

This summarization model can quickly summarize clinical notes, encounters, critical care notes, discharge notes, reports, etc. It is based on a large language model that is finetuned with healthcare-specific additional data curated by medical doctors at John Snow Labs. It can generate summaries up to 512 tokens given an input text (max 1024 tokens).

In [ ]:
text = """ Patient with hypertension, syncope, and spinal stenosis - for recheck.
 (Medical Transcription Sample Report)
 SUBJECTIVE:
 The patient is a 78-year-old female who returns for recheck. She has hypertension. She denies difficulty with chest pain, palpations, orthopnea, nocturnal dyspnea, or edema.
 PAST MEDICAL HISTORY / SURGERY / HOSPITALIZATIONS:
 Reviewed and unchanged from the dictation on 12/03/2003.
 MEDICATIONS:
 Atenolol 50 mg daily, Premarin 0.625 mg daily, calcium with vitamin D two to three pills daily, multivitamin daily, aspirin as needed, and TriViFlor 25 mg two pills daily. She also has Elocon cream 0.1% and Synalar cream 0.01% that she uses as needed for rash."""


data = spark.createDataFrame([[text]]).toDF("text")
data.show(truncate = 60)

+------------------------------------------------------------+
|                                                        text|
+------------------------------------------------------------+
| Patient with hypertension, syncope, and spinal stenosis ...|
+------------------------------------------------------------+



In [ ]:
document_assembler = DocumentAssembler()\
            .setInputCol('text')\
            .setOutputCol('document')

summarizer = MedicalSummarizer.pretrained("summarizer_clinical_jsl", "en", "clinical/models")\
            .setInputCols(['document'])\
            .setOutputCol('summary')\
            .setMaxTextLength(512)\
            .setMaxNewTokens(512)

pipeline = Pipeline(stages=[
            document_assembler,
            summarizer])

model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

result = model.transform(data)

result.show()

summarizer_clinical_jsl download started this may take some time.
[OK!]
+--------------------+--------------------+--------------------+
|                text|            document|             summary|
+--------------------+--------------------+--------------------+
| Patient with hyp...|[{document, 0, 68...|[{document, 0, 24...|
+--------------------+--------------------+--------------------+



In [ ]:
result.select("summary.result").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                                                                                |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[A 78-year-old female with hypertension, syncope, and spinal stenosis returns for recheck. She denies chest pain, palpations, orthopnea, nocturnal dyspnea, or edema. She is on multiple medications and has Elocon cream and Synalar cream for rash.]|
+---

### 📍 LightPipelines

In [ ]:
import textwrap

In [ ]:
text = """The patient is a pleasant 17-year-old gentleman who was playing basketball today in gym. Two hours prior to presentation, he started to fall and someone stepped on his ankle and kind of twisted his right ankle and he cannot bear weight on it now. It hurts to move or bear weight. No other injuries noted. He does not think he has had injuries to his ankle in the past.
SOCIAL HISTORY: He does not drink or smoke.
MEDICAL DECISION MAKING:
He had an x-ray of his ankle that showed a small ossicle versus avulsion fracture of the talonavicular joint on the lateral view. He has had no pain over the metatarsals themselves. This may be a fracture based upon his exam. He does want to have me to put him in a splint. He was given Motrin here. He will be discharged home to follow up with Dr. X from Orthopedics.
DISPOSITION: Crutches and splint were administered here. I gave him a prescription for Motrin and some Darvocet if he needs to length his sleep and if he has continued pain to follow up with Dr. X. Return if any worsening problems."""

light_model = LightPipeline(model)
light_result = light_model.annotate(text)

document_text = textwrap.fill(light_result['document'][0], width=120)
summary_text = textwrap.fill(light_result['summary'][0], width=120)

print("➤ Document :\n{}".format(document_text))
print("\n")
print("➤ Summary : \n{}".format(summary_text))
print("\n")

➤ Document :
The patient is a pleasant 17-year-old gentleman who was playing basketball today in gym. Two hours prior to
presentation, he started to fall and someone stepped on his ankle and kind of twisted his right ankle and he cannot bear
weight on it now. It hurts to move or bear weight. No other injuries noted. He does not think he has had injuries to his
ankle in the past. SOCIAL HISTORY: He does not drink or smoke. MEDICAL DECISION MAKING: He had an x-ray of his ankle
that showed a small ossicle versus avulsion fracture of the talonavicular joint on the lateral view. He has had no pain
over the metatarsals themselves. This may be a fracture based upon his exam. He does want to have me to put him in a
splint. He was given Motrin here. He will be discharged home to follow up with Dr. X from Orthopedics. DISPOSITION:
Crutches and splint were administered here. I gave him a prescription for Motrin and some Darvocet if he needs to length
his sleep and if he has continued pain to foll

### 🚩 summaries from paragraphs in text

In [ ]:
document_assembler = DocumentAssembler()\
            .setInputCol('text')\
            .setOutputCol('document')

sentenceDetector = SentenceDetectorDLModel\
            .pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
            .setInputCols(["document"])\
            .setOutputCol("sentence")\
            .setCustomBounds(["\n"])\
            .setUseCustomBoundsOnly(True)

summarizer = MedicalSummarizer\
            .pretrained("summarizer_clinical_jsl")\
            .setInputCols(['sentence'])\
            .setOutputCol('summary')\
            .setMaxTextLength(512)\
            .setMaxNewTokens(512)

pipeline = Pipeline(stages=[
            document_assembler,
            sentenceDetector,
            summarizer])

model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

Approximate size to download 367.3 KB
[OK!]
summarizer_clinical_jsl download started this may take some time.
[OK!]


In [ ]:
text = """PRESENT ILLNESS: The patient is a 28-year-old, who is status post gastric bypass surgery nearly one year ago. He has lost about 200 pounds and was otherwise doing well until yesterday evening around 7:00-8:00 when he developed nausea and right upper quadrant pain, which apparently wrapped around toward his right side and back. He feels like he was on it but has not done so. He has overall malaise and a low-grade temperature of 100.3. He denies any prior similar or lesser symptoms. His last normal bowel movement was yesterday. He denies any outright chills or blood per rectum.

PHYSICAL EXAMINATION: His temperature is 100.3, blood pressure 129/59, respirations 16, heart rate 84. He is drowsy, but easily arousable and appropriate with conversation. He is oriented to person, place, and situation. He is normocephalic, atraumatic. His sclerae are anicteric. His mucous membranes are somewhat tacky. His neck is supple and symmetric. His respirations are unlabored and clear. He has a regular rate and rhythm. His abdomen is soft. He has diffuse right upper quadrant tenderness, worse focally, but no rebound or guarding. He otherwise has no organomegaly, masses, or abdominal hernias evident. His extremities are symmetrical with no edema. His posterior tibial pulses are palpable and symmetric. He is grossly nonfocal neurologically.

PLAN: He will be admitted and placed on IV antibiotics. We will get an ultrasound this morning. He will need his gallbladder out, probably with intraoperative cholangiogram. Hopefully, the stone will pass this way. Due to his anatomy, an ERCP would prove quite difficult if not impossible unless laparoscopic assisted. Dr. X will see him later this morning and discuss the plan further. The patient understands."""

light_model = LightPipeline(model)
light_result = light_model.annotate(text)

In [ ]:
for i in range(len(light_result['sentence'])):
    document_text = textwrap.fill(light_result['sentence'][i], width=120)
    summary_text = textwrap.fill(light_result['summary'][i], width=120)

    print("➤ Document {}: \n{}".format(i+1, document_text))
    print("\n")
    print("➤ Summary {}: \n{}".format(i+1, summary_text))
    print("\n")

➤ Document 1: 
PRESENT ILLNESS: The patient is a 28-year-old, who is status post gastric bypass surgery nearly one year ago. He has
lost about 200 pounds and was otherwise doing well until yesterday evening around 7:00-8:00 when he developed nausea and
right upper quadrant pain, which apparently wrapped around toward his right side and back. He feels like he was on it
but has not done so. He has overall malaise and a low-grade temperature of 100.3. He denies any prior similar or lesser
symptoms. His last normal bowel movement was yesterday. He denies any outright chills or blood per rectum.


➤ Summary 1: 
A 28-year-old patient who had gastric bypass surgery nearly one year ago developed nausea and right upper quadrant pain,
which wrapped around his right side and back. He has malaise and a low-grade temperature of 100.3. He denies any
previous symptoms and has no other symptoms.


➤ Document 2: 
PHYSICAL EXAMINATION: His temperature is 100.3, blood pressure 129/59, respirations 16, he

### 🚩 setRefineSummary

**We've Made Significant Enhancements To Our Text Summarization Method, Which Now Utilizes A Map-Reduce Approach For Section-Wise Summarization.**

We are excited to announce the integration of new parameters into our `MedicalSummarizer` annotator, empowering users to overcome token limitations and attain heightened flexibility in their medical summarization endeavors. These advanced parameters significantly augment the annotator's functionality, enabling users to generate more accurate and comprehensive summaries of medical documents. By employing a map-reduce approach, the `MedicalSummarizer` efficiently condenses distinct text segments until the desired length is achieved.

The following parameters have been introduced:

- `setRefineSummary`: Set to True for refined summarization with increased computational cost.
- `setRefineSummaryTargetLength`: Define the target length of summarizations in tokens (delimited by whitespace). Effective only when setRefineSummary=True.
- `setRefineChunkSize`: Specify the desired size of refined chunks. Should correspond to the LLM context window size in tokens. Effective only when - `setRefineSummary=True`.
- `setRefineMaxAttempts`: Determine the number of attempts for re-summarizing chunks exceeding the setRefineSummaryTargetLength before discontinuing. Effective only when `setRefineSummary=True`.

These enhancements to the MedicalSummarizer annotator represent our ongoing commitment to providing state-of-the-art tools for healthcare professionals and researchers, facilitating more efficient and accurate medical text analysis.

In [ ]:
document_assembler = DocumentAssembler()\
            .setInputCol('text')\
            .setOutputCol('document')

summarizer = MedicalSummarizer.pretrained("summarizer_clinical_jsl", "en", "clinical/models")\
            .setInputCols(["document"])\
            .setOutputCol("summary")\
            .setMaxTextLength(512)\
            .setMaxNewTokens(512)\
            .setDoSample(True)\
            .setRefineSummary(True)\
            .setRefineSummaryTargetLength(100)\
            .setRefineMaxAttempts(3)\
            .setRefineChunkSize(512)\

pipeline = Pipeline(stages=[
            document_assembler,
            summarizer])

model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

summarizer_clinical_jsl download started this may take some time.
[OK!]


In [ ]:
text = """The patient is a pleasant 17-year-old gentleman who was playing basketball today in gym. Two hours prior to presentation, he started to fall and someone stepped on his ankle and kind of twisted his right ankle and he cannot bear weight on it now. It hurts to move or bear weight. No other injuries noted. He does not think he has had injuries to his ankle in the past.
SOCIAL HISTORY: He does not drink or smoke.
MEDICAL DECISION MAKING:
He had an x-ray of his ankle that showed a small ossicle versus avulsion fracture of the talonavicular joint on the lateral view. He has had no pain over the metatarsals themselves. This may be a fracture based upon his exam. He does want to have me to put him in a splint. He was given Motrin here. He will be discharged home to follow up with Dr. X from Orthopedics.
DISPOSITION: Crutches and splint were administered here. I gave him a prescription for Motrin and some Darvocet if he needs to length his sleep and if he has continued pain to follow up with Dr. X. Return if any worsening problems."""

light_model = LightPipeline(model)
light_result = light_model.annotate(text)

In [ ]:
light_result["summary"]

['An x-ray showed spondylolisis of the ankle in a 17, 17, and 17-months old man who was playing basketball in the gym after slipping. He has no other injury notes, but an anx ray revealed small fracture of the ankle. He requested Moritinos for pain and he was discharged with crutes and Motrinos. The physician gave his sprates & tuxet for resting. He also gave him Motrines, if pain worsenes and advised him on returning for followup.']

In [ ]:
for i in range(len(light_result['document'])):
    document_text = textwrap.fill(light_result['document'][i], width=120)
    summary_text = textwrap.fill(light_result['summary'][i], width=120)

    print("➤ Document {}: \n{}".format(i+1, document_text))
    print("\n")
    print("➤ Summary {}: \n{}".format(i+1, summary_text))
    print("\n")

➤ Document 1: 
The patient is a pleasant 17-year-old gentleman who was playing basketball today in gym. Two hours prior to
presentation, he started to fall and someone stepped on his ankle and kind of twisted his right ankle and he cannot bear
weight on it now. It hurts to move or bear weight. No other injuries noted. He does not think he has had injuries to his
ankle in the past. SOCIAL HISTORY: He does not drink or smoke. MEDICAL DECISION MAKING: He had an x-ray of his ankle
that showed a small ossicle versus avulsion fracture of the talonavicular joint on the lateral view. He has had no pain
over the metatarsals themselves. This may be a fracture based upon his exam. He does want to have me to put him in a
splint. He was given Motrin here. He will be discharged home to follow up with Dr. X from Orthopedics. DISPOSITION:
Crutches and splint were administered here. I gave him a prescription for Motrin and some Darvocet if he needs to length
his sleep and if he has continued pain to fo

In [ ]:
text = """To determine whether a course of low-dose indomethacin therapy, when initiated within 24 hours of birth, would decrease ductal shunting in premature infants who received prophylactic surfactant in the delivery room. Ninety infants, with birth weights of 600 to 1250 gm, were entered into a prospective, randomized, controlled trial to receive either indomethacin, 0.1 mg/kg per dose, or placebo less than 24 hours and again every 24 hours for six doses. Echocardiography was performed on day 1 before treatment and on day 7, 24 hours after treatment. A hemodynamically significant patent ductus arteriosus (PDA) was confirmed with an out-of-study echocardiogram, and the nonresponders were treated with standard indomethacin or ligation. Forty-three infants received indomethacin (birth weight, 915 +/- 209 gm; gestational age, 26.4 +/- 1.6 weeks; 25 boys), and 47 received placebo (birth weight, 879 +/- 202 gm; gestational age, 26.4 +/- 1.8 weeks; 22 boys) (P = not significant). Of 90 infants, 77 (86%) had a PDA by echocardiogram on the first day of life before study treatment; 84% of these PDAs were moderate or large in size in the indomethacin-treated group compared with 93% in the placebo group. Nine of forty indomethacin-treated infants (21%) were study-dose nonresponders compared with 22 (47%) of 47 placebo-treated infants (p < 0.018). There were no significant differences between both groups in any of the long-term outcome variables, including intraventricular hemorrhage, duration of oxygen therapy, endotracheal intubation, duration of stay in neonatal intensive care unit, time to regain birth weight or reach full caloric intake, incidence of bronchopulmonary dysplasia, and survival. No significant differences were noted in the incidence of oliguria, elevated plasma creatinine concentration, thrombocytopenia, pulmonary hemorrhage, or necrotizing enterocolitis. The prophylactic use of low doses of indomethacin, when initiated in the first 24 hours of life in low birth weight infants who receive prophylactic surfactant in the delivery room, decreases the incidence of left-to-right shunting at the level of the ductus arteriosus."""

light_result = light_model.annotate(text)

In [ ]:
light_result["summary"]

["A trial was performed on 90 newborn babies with low birthweight babies who received 0.1mg/kg pro dose indomethadiain or placebo less less then 24 hours for sixdosed. A hemodynamicly significant brevet duplex (DPA), was confirmated by echocordogram, with 47 of the infant'' nonrespirators treated. There was a no significant change from the placebo-controlled study group to re-intravenously-treated newborn infants who had PPAs."]

# **Medical Text Generation**

The objective of this notebook is to explore the Biomedical Generative Pre-trained Transformer (BioGPT) models - `biogpt_chat_jsl` and `biogpt_chat_jsl_conversational_en`, for closed book question answering. These models are pre-trained on large biomedical text data and can generate coherent and relevant responses to biomedical questions.

📖 Learning Objectives:

- Learn how to use the BioGPT models in Spark NLP for closed book question answering tasks, including loading pre-trained models and configuring the pipeline.

- Understand the parameters and options available for the BioGPT models to customize the text generation process based on specific use cases.

# 🔎 MODELS

<div align="center">

| **Index** | **Summarizer Models**        |
|---------------|----------------------|
| 1        | [biogpt_chat_jsl](https://nlp.johnsnowlabs.com/2023/04/12/biogpt_chat_jsl_en.html)     |
| 2          | [biogpt_chat_jsl_conversational](https://nlp.johnsnowlabs.com/2023/04/18/biogpt_chat_jsl_conversational_en.html)       |
| 3      | [biogpt_chat_jsl_conditions](https://nlp.johnsnowlabs.com/2023/05/11/biogpt_chat_jsl_conditions_en.html)    |

</div>

# 	📎🏥 `biogpt_chat_jsl`

This model is based on BioGPT finetuned with medical conversations happening in a clinical settings and can answer clinical questions related to symptoms, drugs, tests, and diseases.

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("documents")

gpt_qa = MedicalTextGenerator().pretrained("biogpt_chat_jsl", "en", "clinical/models")\
    .setInputCols("documents")\
    .setOutputCol("answer")\
    .setMaxNewTokens(299)\
    .setStopAtEos(True)\
    .setDoSample(False)\
    .setTopK(3)\
    .setRandomSeed(42)\
    .setCustomPrompt("question: {DOCUMENT} answer:")

pipeline = Pipeline().setStages([document_assembler, gpt_qa])

empty_data = spark.createDataFrame([[""]]).toDF("text")

model = pipeline.fit(empty_data)

biogpt_chat_jsl download started this may take some time.
[OK!]


In [ ]:
TEXT = "What medications are commonly used to treat emphysema?"

data = spark.createDataFrame([[TEXT]]).toDF("text")

result = model.transform(data)

result.show(truncate=False)

+------------------------------------------------------+------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
result.select("answer.result").show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                        

## **📍 LightPipeline**

In [ ]:
TEXT = "What are the risk factors for developing heart disease?"

light_model = LightPipeline(model)

light_result = light_model.annotate(TEXT)

result_text = light_result["answer"]

In [ ]:
result_text[0].split(" answer: ")

['question: What are the risk factors for developing heart disease ?',
 'Hello, There are several factors that are responsible for the development of heart disease. One of the most important is your cholesterol level. High cholesterol levels are responsible for the development of coronary artery disease. The other factors are blood pressure and smoking. The goal of treatment is to reduce the total and low - density lipoprotein ( LDL ) cholesterol levels. Statins are good cholesterol lowering medications. They help reduce the risk of coronary artery disease by preventing the formation of blood clots. The goal of blood pressure treatment is to reduce the average blood pressure. Regular exercises, weight loss, fruits, vegetables, fish once or twice a week, avoid smoking. The goal of the cholesterol treatment is to bring the LDL level to normal ( less than 100 mg / DL ). The goal of the smoking treatment is to bring the smoking cessation to a significant extent. You should also get your ch

In [ ]:
print("➤ Question: \n{}".format(TEXT))
print("\n")

# Format the text into paragraphs
wrapped_text = textwrap.fill(result_text[0].split(" answer: ")[1], width=120)

print("➤ Answer: \n{}".format(wrapped_text))
print("\n")

➤ Question: 
What are the risk factors for developing heart disease?


➤ Answer: 
Hello, There are several factors that are responsible for the development of heart disease. One of the most important is
your cholesterol level. High cholesterol levels are responsible for the development of coronary artery disease. The
other factors are blood pressure and smoking. The goal of treatment is to reduce the total and low - density lipoprotein
( LDL ) cholesterol levels. Statins are good cholesterol lowering medications. They help reduce the risk of coronary
artery disease by preventing the formation of blood clots. The goal of blood pressure treatment is to reduce the average
blood pressure. Regular exercises, weight loss, fruits, vegetables, fish once or twice a week, avoid smoking. The goal
of the cholesterol treatment is to bring the LDL level to normal ( less than 100 mg / DL ). The goal of the smoking
treatment is to bring the smoking cessation to a significant extent. You should also ge

## 🚩 `setMaxNewTokens`

- This parameter sets the maximum number of new tokens that the GPT model will generate for the output, constraining the length of the generated response and managing the computational cost.

Pipeline with `setMaxNewTokens(128)` and `setMaxNewTokens(299)`

In [ ]:
# Default parameters
gpt_qa = MedicalTextGenerator().pretrained("biogpt_chat_jsl", "en", "clinical/models") \
    .setInputCols("documents") \
    .setOutputCol("answer") \
    .setStopAtEos(True)\
    .setDoSample(False)\
    .setTopK(3) \
    .setRandomSeed(42)\
    .setStopAtEos(True)\
    .setCustomPrompt("question: {DOCUMENT} answer:")

MaxNewTokens = [128, 299]


# Sample question
TEXT = "How can asthma be treated?"


for j in MaxNewTokens:
    print("\nQuestion:", TEXT)
    print("Parameters:")
    print(f"setMaxNewTokens({j}):")

    gpt_qa.setMaxNewTokens(j)
    pipeline = Pipeline().setStages([document_assembler, gpt_qa])

    light_model = LightPipeline(pipeline.fit(spark.createDataFrame([[""]]).toDF("text")))

    result_text = light_model.annotate(TEXT)["answer"][0]

    answer_text = result_text.split(" answer: ")[1]
    wrapped_text = textwrap.fill(answer_text, width=120)
    token_count = len(result_text.split())

    print("➤ Answer:")
    print(wrapped_text)
    print(f"Number of tokens used: {token_count}")
    print("-" * 40)  # Separator line




biogpt_chat_jsl download started this may take some time.
[OK!]

Question: How can asthma be treated?
Parameters:
setMaxNewTokens(128):
➤ Answer:
Hello, Asthma is itself an allergic disease due to cold or dust or pollen or grass etc. irrespective of the triggering
factor. You are not able to get rid from it without taking any medication. You are not able to get control from outside
as it is the only way. You can try the following measures: 1. Improve your air quality by avoiding fine particles (
dust, mite, pollen ). 2. Sugar cane may be the best food to feed your child. 3. Keep your house clean and warm. 4. Use
loose bins / grinders / double joiners / oil skim
Number of tokens used: 109
----------------------------------------

Question: How can asthma be treated?
Parameters:
setMaxNewTokens(299):
➤ Answer:
Hello, Asthma is itself an allergic disease due to cold or dust or pollen or grass etc. irrespective of the triggering
factor. You are not able to get rid from it without taking an

<b><h1><font color='darkred'>!!! ATTENTION !!! </font><h1><b>

<b>before running the following cells, <font color='darkred'>RESTART the COLAB RUNTIME </font> than start your session and go ahead.<b>